<a href="https://colab.research.google.com/github/bborisggg/hw_econometrics/blob/main/SVAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!rm -rf hw_econometrics

In [ ]:
!git clone https://github.com/bborisggg/hw_econometrics

In [ ]:
import numpy as np
import pandas as pd

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA

import matplotlib.pyplot as plt
from matplotlib import gridspec

В данном коде мы непосредственно реализуем модель SVAR для собранных данных

In [ ]:
df = pd.read_excel('hw_econometrics/data.xlsx')